## NN Ensemble
Re-use the three trained networks from 12.A to create an ensemble of nueral networks, with each one running on a different task

In [1]:
import tensorflow as tf
import os

#### Load the data

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

x_train, y_train = x_train, y_train
x_test, y_test = x_test, y_test

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train, x_test = x_train.reshape(-1,28*28), x_test.reshape(-1,28*28)

data = (x_train,y_train,x_test,y_test)

In [3]:
# Create method for getting batches for training

class mini_batches:
    
    def __init__(self, x, y, size):
        self.x = x
        self.y = y
        self.size = size
        self.index = 0
    
    def next_batch(self):
        if self.index + self.size >= len(self.x):            
            batch_x = self.x[self.index:]
            batch_y = self.y[self.index:]
            self.index = 0
            return batch_x, batch_y
        
        batch_x = self.x[self.index:self.index + self.size]
        batch_y = self.y[self.index:self.index + self.size]
        self.index = self.index + self.size
        return batch_x, batch_y

#### Cluster
Only run once!!

In [4]:
tasks = ["localhost:2222", "localhost:2223","localhost:2224"]
jobs = {"local": tasks}
cluster = tf.train.ClusterSpec(jobs)

In [5]:
server1 = tf.train.Server(cluster, job_name="local", task_index=0)
server2 = tf.train.Server(cluster, job_name="local", task_index=1)
server3 = tf.train.Server(cluster, job_name="local", task_index=2)

In [6]:
def run_with_location_trace(sess, op):
    # From https://stackoverflow.com/a/41525764/7832197
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    sess.run(op, options=run_options, run_metadata=run_metadata)
    for device in run_metadata.step_stats.dev_stats:
      print(device.device)
      for node in device.node_stats:
        print("  ", node.node_name)

#### Graphs

In [8]:
tf.reset_default_graph()

In [9]:
from tensorflow.contrib.layers import fully_connected 
from tensorflow.contrib.layers import batch_norm
from tensorflow.contrib.layers import dropout


# Create graph with all three NNs built in, feed to a single output!
        
is_training = tf.placeholder(tf.bool, shape=(), name='is_training')
    
# Inputs for training
X = tf.placeholder(tf.float32, shape=(None,28*28), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')
X_drop = dropout(X,.5, is_training=is_training)
    
# Nueral Network layers
he_init = tf.contrib.layers.variance_scaling_initializer()
bn_params = {'is_training':is_training, 'decay':0.99, 'updates_collections':None}
    
with tf.contrib.framework.arg_scope([fully_connected], weights_initializer=he_init, activation_fn=tf.nn.elu, 
                                            normalizer_fn=batch_norm, normalizer_params=bn_params):
    
    with tf.device("/job:local/task:0"):
        h1_0 = dropout(fully_connected(X_drop, 40, scope='h1_0'))
        h2_0 = dropout(fully_connected(h1_0, 30, scope='h2_0'))
        h3_0 = dropout(fully_connected(h2_0, 20, scope='h3_0'))
        output_0 = fully_connected(h3_0, 10, scope='output_0', activation_fn=None)
        
    with tf.device("/job:local/task:1"):
        h1_1 = dropout(fully_connected(X_drop, 40, scope='h1_1'))
        h2_1 = dropout(fully_connected(h1_1, 30, scope='h2_1'))
        h3_1 = dropout(fully_connected(h2_1, 20, scope='h3_1'))
        output_1 = fully_connected(h3_1, 10, scope='output_1', activation_fn=None)
    
    with tf.device("/job:local/task:2"):
        h1_2 = dropout(fully_connected(X_drop, 40, scope='h1_2'))
        h2_2 = dropout(fully_connected(h1_2, 30, scope='h2_2'))
        h3_2 = dropout(fully_connected(h2_2, 20, scope='h3_2'))
        output_2 = fully_connected(h3_2, 10, scope='output_2', activation_fn=None)
    
# Evaluation of performance
output = tf.add_n([output_0,output_1,output_2])
correct = tf.nn.in_top_k(output, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

#### Sessions

In [17]:
sess1 = tf.Session(server1.target)

In [18]:
value_list = []
value_list.extend(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='h[123]_[012]'))
og_saver = tf.train.Saver(value_list)

In [19]:
# Initiate and restore model

sess1.run(tf.global_variables_initializer())
og_saver.restore(sess1,'./tensorflow/models/12_deep_learning_parallel.ckpt')

In [29]:
# Evaluate accuracy of the model

print(sess1.run(accuracy,feed_dict={X:data[2], y:data[3], is_training:False}, options=run_options, run_metadata=run_metadata))

INFO:tensorflow:Restoring parameters from ./tensorflow/models/12_deep_learning_parallel.ckpt
0.1274


In [30]:
sess1.close()